In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
print(f"TensorFlow Version: {tf.__version__}")

TensorFlow Version: 2.18.0


In [4]:
# same process as other models
data_augmentation = Sequential(
  [
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomShear(x_factor=(0.0, 0.2))  
  ],
  name="data_augmentation",
)

In [5]:
# Parameters from the paper
IMAGE_SIZE = (256, 256)
BATCH_SIZE = 32
NUM_CLASSES = 38 
DATA_DIR = '../input/plantvillage' 

print(f"Image size set to: {IMAGE_SIZE}")
print(f"Data directory is: {DATA_DIR}")

Image size set to: (256, 256)
Data directory is: ../input/plantvillage


In [6]:
train_dir = '../input/plantvillage/PlantVillage/train' # training data path
val_dir = '../input/plantvillage/PlantVillage/val'  # validation data path

#create train dataset
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir, 
    seed=123,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

# create val dataset
val_ds = tf.keras.utils.image_dataset_from_directory(
    val_dir,  
    seed=123,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

class_names = train_ds.class_names
print(f"Loaded {len(class_names)} classes.")
print(f"First 5 classes: {class_names[:5]}...")

Found 43444 files belonging to 38 classes.
Found 10861 files belonging to 38 classes.
Loaded 38 classes.
First 5 classes: ['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Blueberry___healthy']...


In [7]:
# Load the Inception V3 base model (pre-trained on ImageNet)
base_model_inception = tf.keras.applications.InceptionV3(
    input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3),
    include_top=False, 
    weights='imagenet'
)

#Freeze the base model
base_model_inception.trainable = False

# Create your new model on top
model_inception = Sequential([
    # Input layer
    layers.Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)),
    
    # rescaling
    layers.Rescaling(1./255),
    
    # Re-use the same data_augmentation layer
    data_augmentation,
    
    # The Inception V3 base
    base_model_inception,
    
    # The same classifier head
    layers.GlobalAveragePooling2D(),
    layers.Dense(NUM_CLASSES, activation='softmax') 
], name="InceptionV3_Transfer_Learning")

# Compile the model
model_inception.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model_inception.summary()

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "InceptionV3_Transfer_Learning"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)           │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ data_augmentation (Sequential)  │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_v3 (Functional)       │ (None, 6, 6, 2048)     │    21,802,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 38)             │        77,862 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,880,646 (83.47 MB)

 Trainable params: 77,862 (304.15 KB)

 Non-trainable params: 21,802,784 (83.17 MB)

In [9]:
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy', 
    patience=2,              
    restore_best_weights=True 
)

In [11]:
epochs_to_run = 15

history_inception = model_inception.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs_to_run,
    callbacks=[early_stopping_callback] 
)

print("Inception V3 model training complete.")

Epoch 1/15
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 195s 144ms/step - accuracy: 0.7584 - loss: 0.9146 - val_accuracy: 0.8537 - val_loss: 0.4844
Epoch 2/15
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 188s 138ms/step - accuracy: 0.8706 - loss: 0.4191 - val_accuracy: 0.8750 - val_loss: 0.3993
Epoch 3/15
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 188s 138ms/step - accuracy: 0.8901 - loss: 0.3524 - val_accuracy: 0.8956 - val_loss: 0.3328
Epoch 4/15
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 188s 139ms/step - accuracy: 0.8987 - loss: 0.3230 - val_accuracy: 0.8985 - val_loss: 0.3292
Epoch 5/15
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 188s 138ms/step - accuracy: 0.9024 - loss: 0.2989 - val_accuracy: 0.9077 - val_loss: 0.2931
Epoch 6/15
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 188s 138ms/step - accuracy: 0.9072 - loss: 0.2872 - val_accuracy: 0.8930 - val_loss: 0.3286
Epoch 7/15
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 189s 139ms/step - accuracy: 0.9133 - loss: 0.2673 - val_accuracy: 0.9091 - val_loss: 0.2823
Epoch 8/15
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 188s 139ms/step - ac

In [12]:
model_inception.save("/kaggle/working/plant_model_inception.h5")
print("Model saved successfully!") 

Model saved successfully!


# Not doing Fine tuning cuz model achived good accuracy...